In [23]:
import camelot
import pandas as pd
import os



In [24]:

# drive.mount('/content/drive')

# Set the directory containing the PDF files
pdf_dir = '/Users/vamshi/Documents/GenAI/camelot2/content/pdf_files'

# Create the output directory for CSV files
csv_dir = '/Users/vamshi/Documents/GenAI/camelot2/content/csv_files'
os.makedirs(csv_dir, exist_ok=True)

# Process each PDF file in the directory
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]




In [25]:
# Concatenate all tables from all PDF files into a single DataFrame
all_tables_df = pd.DataFrame()
for pdf_file in pdf_files:
    print(f"Processing PDF file: {pdf_file}")
    pdf_path = os.path.join(pdf_dir, pdf_file)
    tables = camelot.read_pdf(pdf_path, pages='all')
    pdf_tables_df = pd.concat([table.df for table in tables])
    all_tables_df = pd.concat([all_tables_df, pdf_tables_df], ignore_index=True)

# Export the combined DataFrame to a single CSV file
csv_file_path = os.path.join(csv_dir, 'multiplehandle.csv')
all_tables_df.to_csv(csv_file_path, index=False)
print(f"Exported all tables to {csv_file_path}")



Processing PDF file: integrated-and-sustainability-report-2023-single-page 3.pdf


/Users/vamshi/Documents/GenAI/camelot2/lib/python3.12/site-packages/camelot/parsers/lattice.py:392: UserWarning: page-1 is image-based, camelot only works on text-based pages.
  warnings.warn(
/Users/vamshi/Documents/GenAI/camelot2/lib/python3.12/site-packages/camelot/parsers/lattice.py:392: UserWarning: page-2 is image-based, camelot only works on text-based pages.
  warnings.warn(
/Users/vamshi/Documents/GenAI/camelot2/lib/python3.12/site-packages/camelot/parsers/lattice.py:392: UserWarning: page-3 is image-based, camelot only works on text-based pages.
  warnings.warn(
/Users/vamshi/Documents/GenAI/camelot2/lib/python3.12/site-packages/camelot/parsers/lattice.py:392: UserWarning: page-18 is image-based, camelot only works on text-based pages.
  warnings.warn(
/Users/vamshi/Documents/GenAI/camelot2/lib/python3.12/site-packages/camelot/parsers/lattice.py:392: UserWarning: page-32 is image-based, camelot only works on text-based pages.
  warnings.warn(
/Users/vamshi/Documents/GenAI/cam

Processing PDF file: RITSL_AR_2022_23_eea6041a79 2.pdf


Invalid stream (index 99) within object 853 0: Stream has ended unexpectedly
Invalid stream (index 99) within object 1053 0: Stream has ended unexpectedly
Invalid stream (index 99) within object 1053 0: Stream has ended unexpectedly
Invalid stream (index 99) within object 1053 0: Stream has ended unexpectedly
Invalid stream (index 99) within object 1053 0: Stream has ended unexpectedly


Processing PDF file: Ambuja_annual_report2022 2.pdf


/Users/vamshi/Documents/GenAI/camelot2/lib/python3.12/site-packages/camelot/parsers/lattice.py:397: UserWarning: No tables found on page-395
  warnings.warn(


Processing PDF file: HM_ASR_2023 2.pdf


DependencyError: PyCryptodome is required for AES algorithm

In [ ]:
# Step 2: Load CSV and Process with Langchain
from langchain_community.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma



In [ ]:
# Load documents from the csv_dir directory
loader = DirectoryLoader(csv_dir)
data = loader.load()



In [ ]:
# Split the text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Generate embeddings for the documents
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed)



In [ ]:
# Set up the LLM and QA chain
llm = Ollama(base_url="http://localhost:11434/", model="llama3")
qachain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())



In [ ]:
# Query the data
question = """can you provide the data matix of "Interest" give me the all the data please provide the what is the value it have"""
response = qachain.invoke({"query": question})
print(response)